# Stable Diffusion - Setup

Stable Diffusion verwendet ein sogenanntes Diffusionsmodell, um aus Text Bilder zu generieren. Das Modell wurde von der LMU München in Zusammenarbeit von Stability AI, CompVis LMU und Runway mit Unterstützung von EleutherAI und LAION entwickelt.

### Konfiguration

Als erstes müssen ein paar Variablen gesetzt werden! Die Variable `SD_ROOT` kann angepasst werden.

In [ ]:
from pathlib import Path

# Where should the webui be downloaded to?
SD_ROOT = Path("../a1111")

# The rest will be set from there
SD_MODEL_PATH = Path(SD_ROOT, "models", "Stable-diffusion")
SD_VAE_PATH = Path(SD_ROOT, "models", "VAE")
SD_LORA_PATH = Path(SD_ROOT, "models", "Lora")

# Only required for remote hosting
# GRADIO_USER = __user__
# GRADIO_PASS = __password__
# !rm -rf {SD_ROOT} && mkdir -p {SD_ROOT}

### Plattformspezifische Einstellungen

Da der Installationsprozess plattformabhängig ist, versucht die folgende Zelle den notwendigen `Installer` zu identifizieren und ob CUDA genutzt werden kann.


In [ ]:
import platform
import subprocess

if platform.system() == "Darwin":
    OS = "macos"
elif platform.system() == "Windows":
    OS = "windows"
else:
    OS = "linux"

INSTALLER = SD_ROOT.joinpath("webui.bat" if OS == "windows" else "webui.sh")

try:
    subprocess.check_output('nvidia-smi')
    CUDA_ARGS = "--xformers"
except Exception:
    CUDA_ARGS = ""

## Automatic1111 Web UI herunterladen

Es ist wieder Zeit ein Projekt zu 'klonen'!

In [ ]:
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git {SD_ROOT}

#### Hat das Klonen funktioniert?

Wir müssen als nächstes eine Funktion definieren, die uns nachfolgend Modelle herunterladen kann.
Außerdem testen wir, ob das Klonen funktioniert hat.

In [ ]:
import requests
import shutil

assert Path(SD_MODEL_PATH).exists()
Path(SD_LORA_PATH).mkdir(parents=True, exist_ok=True)

def download_file(url: str, dest: str, force: bool = False) -> None:
    if Path(dest).exists() and not force:
        return
    with requests.get(url, stream=True) as r:
        with open(dest, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

# Only required for remote hosting
# SD_PERSISTENT_MODELS = Path("/storage/sd/models")
# if Path(SD_PERSISTENT_MODELS).exists():
#     import glob
#     for file in glob.glob(f'{SD_PERSISTENT_MODELS}/*'):
#         print(file)
#         shutil.copy(file, SD_MODEL_PATH)

## Wir brauchen Modelle

Als nächstes benötigen wir Modelle, die wir nutzen können, um Bilder zu generieren. Diese Modelle müssen in den Ordner `<SD_ROOT>/models/Stable-diffusion` gelegt werden. Werden keine Modelle heruntergeladen, lädt A1111 automatisch das Model 1.5 von Stable Diffusion runter. Aber woher können wir Modelle beziehen?

<div style="text-align: center; margin-top: 1rem;">
   <img src="images/huggingface.svg" width="80"><span style="margin-left: 1rem; font-size: 4rem; font-weight:400; vertical-align: bottom;">Hugging Face</span>
</div>

> Hugging Face, the most popular AI tool in Data Science, has been an attention magnet for 316.6 million traffic between September 2022 and August 2023. -- writerbuddy.ai [¹](https://writerbuddy.ai/blog/ai-industry-analysis)

Die wahrscheinlich aktuell größte Community mit der größten Sammlung an GAN-Modellen findet man auf [huggingface.co](https://huggingface.co). Die Seite wird vom US-amerikanischen Unternehmen [Hugging Face, Inc](https://en.wikipedia.org/wiki/Hugging_Face) betreut.


<div style="text-align: center; margin-top: 1rem;">
   <img src="images/civitai.png" width="300">
</div>

> Civitai, a top image generator tool, received 177.2 million visits between September 2022 and August 2023. [...] Interestingly, the average user stayed on the site for about 23 minutes and 19 seconds, almost double the industry average. -- writerbuddy.ai [¹](https://writerbuddy.ai/blog/ai-industry-analysis)

Eine besonders auf bildgenerierende Verfahren fokussierte Community ist unter [civitai.com](https://civitai.com) zu finden. Neben Modellen können Nutzende dort auch Bilder und die zugehörigen Prompts und Einstellungen teilen.

### Eine Auswahl an Modellen

Die folgenden Slides enthalten eine paar Modellvorschläge. Diese beinhalten Stable Diffusion 1.5, SDXL und ein paar Derivate. Die besten Ergebnisse können mit SDXL bzw. Juggernaut XL erreicht werden. Diese Modelle sind jedoch deutlich komplexer als SD 1.5 und benötigen deshalb auch leistungsfähigere Rechner bzw. mehr Rechenzeit. Um die Modelle herunterzuladen, müssen die entsprechende Segmente 'entkommentiert' werden.

In [ ]:
# SD 1.5 an a model spezialized on buildings

# print("Downloading Stable Diffusion 1.5")
# download_file(
#     url="https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors",
#     dest=SD_MODEL_PATH.joinpath("v1-5-pruned-emaonly.safetensors")
# )

# print("Downloading Architecture Real Mix (SD 1.5)")
# download_file(
#     url="https://civitai.com/api/download/models/111612",
#     dest=SD_MODEL_PATH.joinpath("ArchitectureRealMix.safetensors")
# )

In [ ]:
# The base SDXL architecture.
# Even if the SDXL base is not used, the refiner and VAE should be downloaded to be used with SDXL derivates.

# print("Downloading SDXL Base")
# download_file(
#     url="https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors?download=true",
#     dest=SD_MODEL_PATH.joinpath("sd_xl_base_1.0.safetensors")
# )

# print("Downloading SDXL Refiner")
# download_file(
#     url="https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors?download=true",
#     dest=SD_MODEL_PATH.joinpath("sd_xl_refiner_1.0.safetensors")
# )

# print("Downloading SDXL Base VAE")
# download_file(
#     url="https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors?download=true",
#     dest=SD_VAE_PATH.joinpath("sdxl_vae.safetensors")
# )

# print("Downloading Juggernaut XL (SDXL)")
# download_file(
#     url="https://civitai.com/api/download/models/198530?type=Model&format=SafeTensor&size=full&fp=fp16",
#     dest=SD_MODEL_PATH.joinpath("JuggernautXL.safetensors"),
# )

In [ ]:
# These are two examples for "Low Rank Adaption" weights which can be combined with a variety of SDXL models

# print("Downloading Amanda Sage LoRA")
# download_file(
#     url="https://civitai.com/api/download/models/210626?type=Model&format=SafeTensor",
#     dest=SD_LORA_PATH.joinpath("AmandaSage.safetensors")
# )

# print("Downloading DetailTweaker LoRA")
# download_file(
#     url="https://civitai.com/api/download/models/135867?type=Model&format=SafeTensor",
#     dest=SD_LORA_PATH.joinpath("DetailTweakerXL.safetensors")
# )

## Anmerkungen zu Modelformaten

Modelle, VAEs, LoRAs etc. kommen in verschiedenen Formaten. Dateien mit der Endung `.ckpt`, `.pt`, `.pth` sind im üblichen Dateiformat von [PyTorch](https://pytorch.org/) gespeichert. Diese basiert auf dem Serialisierungsprotokoll 'Pickle' (`.pkl`) von Python. Pickle-Dateien können beim Laden schadhaften Python-Code ausführen. Deshalb wird empfohlen, 'SafeTensors'-Modelle mit dem Suffix `.safetensors` zu nutzen.

### Automatic1111 starten

Sind Modelle, VAEs und LoRAs heruntergeladen, folgt der eigentliche und einzige Installationsschritt.
Dazu muss nur folgender Code ausgeführt werden.

In [ ]:
%env COMMANDLINE_ARGS {CUDA_ARGS}

# Only required for remote hosting
# %env COMMANDLINE_ARGS -f --xformers --share --gradio-auth {GRADIO_USER}:{GRADIO_PASS}

%env TRANSFORMERS_CACHE {SD_ROOT}/.cache/hf
%env PIP_CACHE_DIR {SD_ROOT}/.pipcache
%env SD_ROOT {SD_ROOT}
%env INSTALLER {INSTALLER}

!cd ${SD_ROOT} && ${INSTALLER} ${COMMANDLINE_ARGS}